In [78]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Conv2D, UpSampling2D, InputLayer, Lambda
import tensorflow.keras.backend as K

In [41]:
vgg19_cl = keras.applications.VGG19()
vgg19_cl.trainable = False
vgg19_cl.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [42]:
model_inp = keras.Sequential(name="conv_imag")
model_inp.add(InputLayer(input_shape=(224, 224, 1)))
model_inp.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model_inp.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model_inp.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model_inp.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
model_inp.summary()

Model: "conv_imag"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 224, 224, 64)      640       
                                                                 
 conv2d_13 (Conv2D)          (None, 112, 112, 64)      36928     
                                                                 
 conv2d_14 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_15 (Conv2D)          (None, 56, 56, 128)       147584    
                                                                 
 conv2d_16 (Conv2D)          (None, 56, 56, 256)       295168    
                                                                 
 conv2d_17 (Conv2D)          (None, 28, 28, 256)       590080    
                                                                 
 conv2d_18 (Conv2D)          (None, 28, 28, 512)       11

In [43]:
model_out = keras.Sequential()
model_out.add(InputLayer(input_shape=(28, 28, 1025)))
model_out.add(UpSampling2D((2, 2)))
model_out.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_out.add(UpSampling2D((2, 2)))
model_out.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model_out.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model_out.add(UpSampling2D((2, 2)))
model_out.summary()

In [121]:
inp_conv = InputLayer((224, 224, 1))


def func(x1, x2):
    """

    :param x1:  shape = (28, 28, 1024)
    :param x2: shape = (1000, )
    :return: x3 shape (28, 28, 1026)
    """
    temp_zero = np.zeros(shape=(28*28*2))
    temp_zero[:1000] = x2
    x2 = np.reshape(temp_zero,newshape=(28,28,2))
    return np.reshape(np.append(x1,x2),newshape=(28,28,1026))

x1 = np.array([[[i for i in range(1024)] for j in range(28)]for _ in range(28)])
x2 = np.array([0 if i != 10 else 1 for i in range(1000)])


func(x1,x2).shape



tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(224, 224), dtype=float32) KerasTensor(type_spec=TensorSpec(shape=(None, 224, 1), dtype=tf.float32, name=None), name='tf.__operators__.getitem_35/strided_slice:0', description="created by layer 'tf.__operators__.getitem_35'")
(None, 224, 224)
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224), dtype=tf.float32, name=None), name='tf.__operators__.add_11/AddV2:0', description="created by layer 'tf.__operators__.add_11'")


ValueError: Exception encountered when calling layer "vgg19" (type Functional).

Input 0 of layer "block1_conv1" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 224, 224)

Call arguments received by layer "vgg19" (type Functional):
  • inputs=tf.Tensor(shape=(None, 224, 224), dtype=float32)
  • training=None
  • mask=None